In [16]:
import pdfplumber
import pandas as pd
import string
from IPython.display import display
from tabulate import tabulate
import os
from openpyxl import load_workbook
from openpyxl.styles.alignment import Alignment
from openpyxl.styles import Alignment
import openpyxl
import xlwings as xw
import datetime
import calendar
import shutil
# !pip3 install openpyxl==2.5.14 

In [3]:
my_path = "E:\\nadupallisriniv\\"
os.chdir(my_path)
os.getcwd()

'E:\\nadupallisriniv'

In [5]:
All_files = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
# All_files

In [10]:
YearandMonths = {}
replacers = {f'{i}\n':'' for i in string.ascii_letters}
replacers.update({"\n":"",None:" "})
for f in All_files:
    pdf = pdfplumber.open(f)
    first_page = pdf.pages[0]
    All_Tables = first_page.extract_tables()
    Details3_1 = pd.DataFrame(columns=[i.replace("\n"," ") for i in All_Tables[2][0]])
    for i in range(len(All_Tables[2][1:])):
        Details3_1.loc[i] = All_Tables[2][1:][i]
    Details3_1 = Details3_1.replace(replacers,regex=True)
    values = {}
    for i in range(len(Details3_1.columns[1:-1])):
        values[Details3_1.columns[1:-1][i]] = float(Details3_1[Details3_1.columns[1:-1][i]][0])
    values
    YearandMonths.setdefault(All_Tables[0][0][1], {})
    YearandMonths[All_Tables[0][0][1]][All_Tables[0][1][1]] = values

In [15]:
os.mkdir("E:\\Financial_Years")

In [17]:
shutil.move("C:\\Users\\aakas\\Aakash Nadupalli ipynb files\\PDF_PLUMBER\\GST FY 2019-20.xlsx", "E:\\Financial_Years")

'E:\\Financial_Years\\GST FY 2019-20.xlsx'

In [62]:
for y in list(YearandMonths.keys()):
    df =   xw.Book('E:\\Financial_Years\\GST_FY_Format_copy.xlsx')
    excel_app = xw.apps.active
    sn = df.sheets[0]
    sn.range('A1').value = "SAI ENTERPRISES"
    sn['A2'].value = "GST  STATEMENT FOR THE PERIOD OF 01-04-{} TO 31-03-20{} GSTR 3B".format(y[0:4],y[5:])
    sn.range('I8').value = "OUTPUT TAXABLE VALUE"
    sn.range('K8').value = "IGST"
    sn.range('L8').value = "CGST"
    sn.range('M8').value = "SGST"
    for clmns in ['I','K','L','M']:
        for numbs in range(11,23):
            jn = ''.join([clmns,str(numbs)])
            sn.range(jn).value = float(0.0)
    for j in list(YearandMonths[y].keys()):
        if j in list(YearandMonths['2021-22'].keys()):
            for s in range(1,30):
                dto = datetime.datetime.strptime(j[4:],"%b")
                cmn = calendar.month_name[dto.month]
                if  cmn == sn.range(s,1).value:
                    for (k,m) in zip(['I','K','L','M'],list(YearandMonths[y][j].values())):
                        cb = ''.join([k,sn.range(s,1).address.replace('$','')[1:]])
                        sn.range(cb).value = m
        else:
            for s in range(1,30):
                if j == (sn.range(s,1).value):
                    for (k,m) in zip(['I','K','L','M'],list(YearandMonths[y][j].values())):
                        cb = ''.join([k,sn.range(s,1).address.replace('$','')[1:]])
                        sn.range(cb).value = m
    df.save('E:\\Financial_Years\\GST_FY_{}.xlsx'.format(y))
    excel_app.quit()